In [17]:
import pandas as pd

# Login Information
### Read secrets

In [3]:
import json
f = open('../secrets/splunk.json')
k = json.load(f)

In [4]:
HOST  = k['HOST']
PORT  = k['PORT']
TOKEN = k['TOKEN']
#USER = k['USER']
#PASS = k['PASS']

### Write secrets

In [5]:
#k = {
#    "HOST"  : HOST,
#    "PORT"  : PORT,
#    "TOKEN" : TOKEN,
#    "USER"  : USER,
#    "PASS"  : PASS
#}

#import json
#with open('../secrets/splunk.json', 'w') as fp:
#    json.dump(k, fp)

# Fetch data from Splunk

In [6]:
import splunklib.client as client
import splunklib.results as results

from time import sleep

### Connect to Splunk

In [7]:
service = client.connect(
    host     = HOST,
    port     = PORT,
    token    = TOKEN,
#    username = USER,
#    password = PASS
)

### Search

In [12]:
SEARCH = "inputlookup iris-data.csv"

# Run search job on connected service
job = service.jobs.create(
    SEARCH
)

# Wait for the search to end
while not job.is_done():
    sleep(0.2)

# Read reponse
data = results.JSONResultsReader(job.results(
    count = 0,           # No limitations for results. Default: 100
    output_mode='json'
))

# Save response lines into array
vec = []
for result in data:
    if isinstance(result, results.Message) :
        print('%s: %s' % (result.type, result.message))
    elif isinstance(result, dict):
        vec.append(result)
print(data.is_preview)

INFO: Successfully read lookup file '/opt/splunk/etc/users/admin/search/lookups/iris-data.csv'.
False


### Structure

In [20]:
df = pd.DataFrame(vec).set_index('Id')

In [21]:
df.head()

,PetalLengthCm,PetalWidthCm,SepalLengthCm,SepalWidthCm,Species
Id,,,,,
51,4.7,1.4,7.0,3.2,Iris-versicolor
52,4.5,1.5,6.4,3.2,Iris-versicolor
53,4.9,1.5,6.9,3.1,Iris-versicolor
54,4.0,1.3,5.5,2.3,Iris-versicolor
55,4.6,1.5,6.5,2.8,Iris-versicolor
